In [1]:
from func import *
from PIL import Image
import numpy as np
import imageio
import matplotlib.pyplot as plt
import random as rnd
import os
import cv2
import bottleneck as bn
from scipy.signal import argrelextrema
from scipy.interpolate import CubicSpline
from keras.models import Sequential
#нейронная сеть, слои которой соединены друг с другом

from keras.layers import Dense
#тип слоев, когда нейроны предыдущего уровня соединяются со всеми нейронами следующего уровня

from keras.utils import np_utils
#утилиты для numpy

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [2]:
rnd.seed(30)

In [6]:
import tensorflow
import keras
tensorflow.__version__

'1.13.1'

In [7]:
keras.__version__

'2.2.4'

In [62]:

data_let = []
data_num = []
ans_let = []
ans_num = []

In [63]:
%%time
path = os.path.abspath("")
itera = 0
for i in os.listdir(path + "/" + "parse_img_no_counturs"):
    pic = imageio.imread(path + "/" + "parse_img_no_counturs/" + i)
    if (int(i[-5:-4]) > 0) and (int(i[-5:-4]) <= 3):
        data_num.append(pic.reshape((-1)))
        ans_num.append(i[int(i[-5:-4])])
    else:
        data_let.append(pic.reshape((-1)))
        ans_let.append(i[int(i[-5:-4])])
    
    itera += 1
    

CPU times: user 13.5 s, sys: 657 ms, total: 14.1 s
Wall time: 14.1 s


In [64]:
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(data_let, ans_let, test_size=0.3)
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(data_num, ans_num, test_size=0.3)
len(y_test_n)

9738

In [65]:
Y = np_utils.to_categorical(y_train_n)


letters = np.unique(y_train_l)
ln = letters.shape
Y1 = np.array([[0 for i in range(ln[0])] for j in range(len(y_train_l))])
for i in range(Y1.shape[0]):
    Y1[i,(y_train_l[i] == letters).argmax()] = 1
print(Y1.shape)

(22722, 12)


In [66]:
model_com = Sequential()
model_let = Sequential()
model_num = Sequential()

In [67]:
model_num.add(Dense(800, input_dim=3468, init="normal", activation="tanh"))
model_num.add(Dense(800, input_dim=800, init="normal", activation="relu"))
model_num.add(Dense(1000, input_dim=800, init="normal", activation="selu"))

model_let.add(Dense(800, input_dim=3468, init="normal", activation="tanh"))

/home/evgeny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(800, input_dim=3468, activation="tanh", kernel_initializer="normal")`
  """Entry point for launching an IPython kernel.
/home/evgeny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(800, input_dim=800, activation="relu", kernel_initializer="normal")`
  
/home/evgeny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1000, input_dim=800, activation="selu", kernel_initializer="normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/evgeny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(800, input_dim=3468, activation="tanh", kernel_initializer="normal")`
  """


In [68]:
model_num.add(Dense(9, init="normal", activation="softmax"))
model_let.add(Dense(Y1.shape[1], init="normal", activation="softmax"))

/home/evgeny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(9, activation="softmax", kernel_initializer="normal")`
  """Entry point for launching an IPython kernel.
/home/evgeny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, activation="softmax", kernel_initializer="normal")`
  


In [69]:
model_let.compile(loss="categorical_crossentropy", 
               optimizer="SGD", 
               metrics=["accuracy"])

In [70]:
model_num.compile(loss="categorical_crossentropy", 
               optimizer="SGD", 
               metrics=["accuracy"])


In [71]:
print(model_let.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 800)               2775200   
_________________________________________________________________
dense_50 (Dense)             (None, 12)                9612      
Total params: 2,784,812
Trainable params: 2,784,812
Non-trainable params: 0
_________________________________________________________________
None


In [72]:
%%time
model_let.fit(np.array(X_train_l), Y1, batch_size=15000, epochs=750,
         verbose=1)

Epoch 1/750
22722/22722 [==============================] - 1s 37us/step - loss: 2.9859 - acc: 0.1028
Epoch 2/750
22722/22722 [==============================] - 0s 16us/step - loss: 2.2544 - acc: 0.2242
Epoch 3/750
22722/22722 [==============================] - 0s 15us/step - loss: 2.0443 - acc: 0.3652
Epoch 4/750
22722/22722 [==============================] - 0s 15us/step - loss: 1.9911 - acc: 0.3834
Epoch 5/750
22722/22722 [==============================] - 0s 15us/step - loss: 1.9062 - acc: 0.4253
Epoch 6/750
22722/22722 [==============================] - 0s 15us/step - loss: 1.8520 - acc: 0.4684
Epoch 7/750
22722/22722 [==============================] - 0s 15us/step - loss: 1.8454 - acc: 0.4640
Epoch 8/750
22722/22722 [==============================] - 0s 15us/step - loss: 1.7650 - acc: 0.5097
Epoch 9/750
22722/22722 [==============================] - 0s 15us/step - loss: 1.7857 - acc: 0.4611
Epoch 10/750
22722/22722 [==============================] - 0s 16us/step - loss: 1.7550 - a

22722/22722 [==============================] - 0s 16us/step - loss: 0.6531 - acc: 0.8432
Epoch 82/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.6552 - acc: 0.8469
Epoch 83/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.6431 - acc: 0.8504
Epoch 84/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.6284 - acc: 0.8497
Epoch 85/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.6273 - acc: 0.8508
Epoch 86/750
22722/22722 [==============================] - 0s 17us/step - loss: 0.6240 - acc: 0.8507
Epoch 87/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.6256 - acc: 0.8501
Epoch 88/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.6224 - acc: 0.8528
Epoch 89/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.6146 - acc: 0.8542
Epoch 90/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.6106 - acc: 

22722/22722 [==============================] - 0s 15us/step - loss: 0.4588 - acc: 0.8830
Epoch 161/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.4575 - acc: 0.8856
Epoch 162/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.4530 - acc: 0.8850
Epoch 163/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.4523 - acc: 0.8869
Epoch 164/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.4491 - acc: 0.8862
Epoch 165/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.4455 - acc: 0.8864
Epoch 166/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.4439 - acc: 0.8881
Epoch 167/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.4415 - acc: 0.8910
Epoch 168/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.4432 - acc: 0.8887
Epoch 169/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.442

22722/22722 [==============================] - 0s 15us/step - loss: 0.3647 - acc: 0.9085
Epoch 240/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3627 - acc: 0.9084
Epoch 241/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3637 - acc: 0.9079
Epoch 242/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3604 - acc: 0.9068
Epoch 243/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3617 - acc: 0.9081
Epoch 244/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3582 - acc: 0.9104
Epoch 245/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3544 - acc: 0.9111
Epoch 246/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3573 - acc: 0.9097
Epoch 247/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3545 - acc: 0.9089
Epoch 248/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.354

22722/22722 [==============================] - 0s 15us/step - loss: 0.3068 - acc: 0.9245
Epoch 319/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3127 - acc: 0.9219
Epoch 320/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3065 - acc: 0.9235
Epoch 321/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3071 - acc: 0.9227
Epoch 322/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3108 - acc: 0.9220
Epoch 323/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3073 - acc: 0.9236
Epoch 324/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3027 - acc: 0.9246
Epoch 325/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3040 - acc: 0.9230
Epoch 326/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.3012 - acc: 0.9245
Epoch 327/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.300

22722/22722 [==============================] - 0s 16us/step - loss: 0.2703 - acc: 0.9286
Epoch 398/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.2685 - acc: 0.9328
Epoch 399/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2708 - acc: 0.9326
Epoch 400/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2664 - acc: 0.9342
Epoch 401/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2670 - acc: 0.9326
Epoch 402/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2643 - acc: 0.9339
Epoch 403/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2629 - acc: 0.9363
Epoch 404/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.2633 - acc: 0.9356
Epoch 405/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2645 - acc: 0.9348
Epoch 406/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.262

22722/22722 [==============================] - 0s 15us/step - loss: 0.2388 - acc: 0.9416
Epoch 477/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2403 - acc: 0.9405
Epoch 478/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2399 - acc: 0.9402
Epoch 479/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2377 - acc: 0.9409
Epoch 480/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2345 - acc: 0.9415
Epoch 481/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2364 - acc: 0.9419
Epoch 482/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2354 - acc: 0.9413
Epoch 483/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2340 - acc: 0.9422
Epoch 484/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2357 - acc: 0.9423
Epoch 485/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.234

22722/22722 [==============================] - 0s 16us/step - loss: 0.2194 - acc: 0.9460
Epoch 556/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2154 - acc: 0.9475
Epoch 557/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2168 - acc: 0.9473
Epoch 558/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2158 - acc: 0.9458
Epoch 559/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2144 - acc: 0.9475
Epoch 560/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2150 - acc: 0.9478
Epoch 561/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2159 - acc: 0.9468
Epoch 562/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2139 - acc: 0.9471
Epoch 563/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.2125 - acc: 0.9471
Epoch 564/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.210

22722/22722 [==============================] - 0s 15us/step - loss: 0.1999 - acc: 0.9519
Epoch 635/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.1993 - acc: 0.9522
Epoch 636/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.1977 - acc: 0.9528
Epoch 637/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.1976 - acc: 0.9520
Epoch 638/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.1982 - acc: 0.9519
Epoch 639/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.1984 - acc: 0.9517
Epoch 640/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.1974 - acc: 0.9504
Epoch 641/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.1966 - acc: 0.9511
Epoch 642/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.1972 - acc: 0.9499
Epoch 643/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.194

22722/22722 [==============================] - 0s 15us/step - loss: 0.1822 - acc: 0.9561
Epoch 714/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.1830 - acc: 0.9560
Epoch 715/750
22722/22722 [==============================] - 0s 15us/step - loss: 0.1817 - acc: 0.9555
Epoch 716/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.1821 - acc: 0.9556
Epoch 717/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.1832 - acc: 0.9544
Epoch 718/750
22722/22722 [==============================] - 0s 17us/step - loss: 0.1821 - acc: 0.9564
Epoch 719/750
22722/22722 [==============================] - 0s 17us/step - loss: 0.1818 - acc: 0.9563
Epoch 720/750
22722/22722 [==============================] - 0s 17us/step - loss: 0.1801 - acc: 0.9557
Epoch 721/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.1808 - acc: 0.9567
Epoch 722/750
22722/22722 [==============================] - 0s 16us/step - loss: 0.185

In [19]:
%%time
model_num.fit(np.array(X_train_n), Y, batch_size=15000, epochs=750,
         verbose=1)

Epoch 1/750
22722/22722 [==============================] - 1s 34us/step - loss: 4.6701 - acc: 0.1084
Epoch 2/750
22722/22722 [==============================] - 1s 22us/step - loss: 6.5318 - acc: 0.1748
Epoch 3/750
22722/22722 [==============================] - 1s 22us/step - loss: 7.2865 - acc: 0.1580
Epoch 4/750
22722/22722 [==============================] - 0s 22us/step - loss: 8.2556 - acc: 0.2092
Epoch 5/750
22722/22722 [==============================] - 0s 22us/step - loss: 8.3695 - acc: 0.1176
Epoch 6/750
22722/22722 [==============================] - 0s 22us/step - loss: 9.4792 - acc: 0.1687
Epoch 7/750
22722/22722 [==============================] - 0s 22us/step - loss: 10.0154 - acc: 0.1347
Epoch 8/750
22722/22722 [==============================] - 0s 22us/step - loss: 10.5937 - acc: 0.1365
Epoch 9/750
22722/22722 [==============================] - 0s 22us/step - loss: 11.2256 - acc: 0.1098
Epoch 10/750
22722/22722 [==============================] - 0s 22us/step - loss: 11.6424

22722/22722 [==============================] - 1s 23us/step - loss: 0.4105 - acc: 0.8946
Epoch 82/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.3919 - acc: 0.8983
Epoch 83/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.3951 - acc: 0.9001
Epoch 84/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.3949 - acc: 0.8987
Epoch 85/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.3760 - acc: 0.9056
Epoch 86/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.3624 - acc: 0.9094
Epoch 87/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.3632 - acc: 0.9093
Epoch 88/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.3577 - acc: 0.9134
Epoch 89/750
22722/22722 [==============================] - 0s 21us/step - loss: 0.3613 - acc: 0.9107
Epoch 90/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.3569 - acc: 

22722/22722 [==============================] - 0s 22us/step - loss: 0.2339 - acc: 0.9445
Epoch 161/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.2348 - acc: 0.9446
Epoch 162/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.2291 - acc: 0.9443
Epoch 163/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.2268 - acc: 0.9465
Epoch 164/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.2324 - acc: 0.9454
Epoch 165/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.2348 - acc: 0.9420
Epoch 166/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.2372 - acc: 0.9438
Epoch 167/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.2274 - acc: 0.9462
Epoch 168/750
22722/22722 [==============================] - 1s 23us/step - loss: 0.2230 - acc: 0.9473
Epoch 169/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.224

22722/22722 [==============================] - 1s 22us/step - loss: 0.1780 - acc: 0.9586
Epoch 240/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.1812 - acc: 0.9571
Epoch 241/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.1848 - acc: 0.9563
Epoch 242/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1827 - acc: 0.9580
Epoch 243/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1803 - acc: 0.9588
Epoch 244/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1799 - acc: 0.9591
Epoch 245/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1834 - acc: 0.9570
Epoch 246/750
22722/22722 [==============================] - 0s 21us/step - loss: 0.1807 - acc: 0.9593
Epoch 247/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.1786 - acc: 0.9580
Epoch 248/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.175

22722/22722 [==============================] - 0s 22us/step - loss: 0.1499 - acc: 0.9671
Epoch 319/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.1523 - acc: 0.9662
Epoch 320/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1507 - acc: 0.9658
Epoch 321/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1543 - acc: 0.9648
Epoch 322/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1491 - acc: 0.9656
Epoch 323/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1517 - acc: 0.9650
Epoch 324/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.1504 - acc: 0.9655
Epoch 325/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1450 - acc: 0.9665
Epoch 326/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1465 - acc: 0.9665
Epoch 327/750
22722/22722 [==============================] - 0s 21us/step - loss: 0.143

22722/22722 [==============================] - 0s 22us/step - loss: 0.1351 - acc: 0.9689
Epoch 398/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1339 - acc: 0.9695
Epoch 399/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1316 - acc: 0.9697
Epoch 400/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1319 - acc: 0.9700
Epoch 401/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1282 - acc: 0.9709
Epoch 402/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1283 - acc: 0.9710
Epoch 403/750
22722/22722 [==============================] - 0s 21us/step - loss: 0.1310 - acc: 0.9709
Epoch 404/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1284 - acc: 0.9715
Epoch 405/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1283 - acc: 0.9711
Epoch 406/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.127

22722/22722 [==============================] - 0s 22us/step - loss: 0.1162 - acc: 0.9748
Epoch 477/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1157 - acc: 0.9746
Epoch 478/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1156 - acc: 0.9754
Epoch 479/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1188 - acc: 0.9743
Epoch 480/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1171 - acc: 0.9743
Epoch 481/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1144 - acc: 0.9747
Epoch 482/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1164 - acc: 0.9753
Epoch 483/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1167 - acc: 0.9745
Epoch 484/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1144 - acc: 0.9755
Epoch 485/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.114

22722/22722 [==============================] - 0s 22us/step - loss: 0.1029 - acc: 0.9776
Epoch 556/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1040 - acc: 0.9774
Epoch 557/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.1031 - acc: 0.9777
Epoch 558/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1045 - acc: 0.9775
Epoch 559/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1035 - acc: 0.9782
Epoch 560/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1033 - acc: 0.9772
Epoch 561/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1047 - acc: 0.9782
Epoch 562/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1028 - acc: 0.9773
Epoch 563/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.1016 - acc: 0.9782
Epoch 564/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.103

22722/22722 [==============================] - 0s 22us/step - loss: 0.0967 - acc: 0.9796
Epoch 635/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.0960 - acc: 0.9790
Epoch 636/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.0929 - acc: 0.9791
Epoch 637/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.0958 - acc: 0.9798
Epoch 638/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.0958 - acc: 0.9794
Epoch 639/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.0958 - acc: 0.9792
Epoch 640/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.0958 - acc: 0.9792
Epoch 641/750
22722/22722 [==============================] - 1s 24us/step - loss: 0.0935 - acc: 0.9802
Epoch 642/750
22722/22722 [==============================] - 1s 23us/step - loss: 0.0950 - acc: 0.9797
Epoch 643/750
22722/22722 [==============================] - 1s 23us/step - loss: 0.099

22722/22722 [==============================] - 1s 23us/step - loss: 0.0856 - acc: 0.9818
Epoch 714/750
22722/22722 [==============================] - 1s 24us/step - loss: 0.0863 - acc: 0.9820
Epoch 715/750
22722/22722 [==============================] - 1s 23us/step - loss: 0.0918 - acc: 0.9795
Epoch 716/750
22722/22722 [==============================] - 1s 23us/step - loss: 0.0900 - acc: 0.9805
Epoch 717/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.0853 - acc: 0.9813
Epoch 718/750
22722/22722 [==============================] - 0s 22us/step - loss: 0.0865 - acc: 0.9818
Epoch 719/750
22722/22722 [==============================] - 1s 23us/step - loss: 0.0852 - acc: 0.9813
Epoch 720/750
22722/22722 [==============================] - 1s 23us/step - loss: 0.0852 - acc: 0.9820
Epoch 721/750
22722/22722 [==============================] - 1s 23us/step - loss: 0.0856 - acc: 0.9822
Epoch 722/750
22722/22722 [==============================] - 1s 22us/step - loss: 0.085

In [73]:
pred_let = model_let.predict(np.array(X_test_l))
pred_num = model_num.predict(np.array(X_test_n))
Y = np.array(y_test_n)
Y1 = np.array(y_test_l)
pred_let.shape
Y1

array(['K', 'O', 'Y', ..., 'C', 'C', 'K'], dtype='<U1')

In [74]:
suma = 0
for i in range(pred_let.shape[0]):
    if letters[(pred_let.argmax(axis=1)[i])] == (Y1[i]):
        suma += 1

In [75]:
print("SCORE_LET", suma/pred_let.shape[0])

SCORE_LET 0.9387964674471144


In [23]:
suma = 0
for i in range(pred_num.shape[0]):
    if int(pred_num.argmax(axis=1)[i]) == int(Y[i]):
        suma += 1

In [24]:
print("SCORE_NUM", suma/pred_num.shape[0])

SCORE_NUM 0.9630314232902033
